In [45]:
import numpy as np
import json
from os import listdir
from os.path import isfile, join
from sklearn.model_selection import train_test_split
import torch
import zipfile
import bz2
from transformers import DistilBertTokenizer, DistilBertModel
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [3]:
#https://ai.stanford.edu/~amaas/data/sentiment/

In [10]:
data_main_path = "../data/movie_reviews/aclImdb/"

In [16]:
train_pos_files = [f for f in listdir(data_main_path + "train/pos/") if isfile(join(data_main_path + "train/pos/", f))]
train_neg_files = [f for f in listdir(data_main_path + "train/neg/") if isfile(join(data_main_path + "train/neg/", f))]
test_pos_files = [f for f in listdir(data_main_path + "test/pos/") if isfile(join(data_main_path + "test/pos/", f))]
test_neg_files = [f for f in listdir(data_main_path + "test/neg/") if isfile(join(data_main_path + "test/neg/", f))]
print(train_pos_files[0:10])

['4715_9.txt', '12390_8.txt', '8329_7.txt', '9063_8.txt', '3092_10.txt', '9865_8.txt', '6639_10.txt', '10460_10.txt', '10331_10.txt', '11606_10.txt']


In [18]:
train_pos_all = []
for i in train_pos_files:
    with open(data_main_path + "train/pos/" + i) as f:
        train_pos_all.append(f.read())

In [22]:
train_neg_all = []
for i in train_neg_files:
    with open(data_main_path + "train/neg/" + i) as f:
        train_neg_all.append(f.read())

In [23]:
test_pos_all = []
for i in test_pos_files:
    with open(data_main_path + "test/pos/" + i) as f:
        test_pos_all.append(f.read())

In [24]:
test_neg_all = []
for i in test_neg_files:
    with open(data_main_path + "test/neg/" + i) as f:
        test_neg_all.append(f.read())

In [25]:
len(train_pos_all), len(train_neg_all), len(test_pos_all), len(test_neg_all)

(12500, 12500, 12500, 12500)

In [30]:
all_data = train_pos_all + train_neg_all + test_pos_all + test_neg_all
all_label = [2 for _ in range(len(train_pos_all))] + [1 for _ in range(len(train_neg_all))] + [2 for _ in range(len(test_pos_all))] + [1 for _ in range(len(test_neg_all))] 

In [31]:
len(all_data), len(all_label)

(50000, 50000)

In [34]:
X_train_val, X_test, y_train_val, y_test = train_test_split(all_data, all_label, test_size=0.33, random_state=7)

In [36]:
len(X_train_val), len(y_train_val)

(33500, 33500)

In [41]:
print(sum([x==1 for x in y_train_val[:2000]])/len(y_train_val[:2000])) # data is balanced
print(sum([x==1 for x in y_test[:2000]])/len(y_test[:2000])) # data is balanced

0.501
0.502


In [40]:
def tokenize_encode_sentences(tokenizer, model, input_sentences, output_path):
    output = np.zeros([len(input_sentences), 768])
    for i, x in enumerate(input_sentences): 
        output[i] = tokenize_encode_sentences_sample(tokenizer, model, [x])
    np.save(output_path, output)
    return output

def tokenize_encode_sentences_sample(tokenizer, model, input_sentences):
    encoded_input = tokenizer(input_sentences, return_tensors = 'pt', truncation = True, padding = True)
    output = model(**encoded_input)[0][:,0,:].detach().numpy()
    return output

In [46]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
bert_train = tokenize_encode_sentences(tokenizer, model, list(X_train_val[:2000]), "../outputs/" + "encoded_stf_movie_train_2000")
bert_test = tokenize_encode_sentences(tokenizer, model, list(X_test[:2000]), "../outputs/" + "encoded_stf_movie_train_2000")